# Estudo de Parcelamento das Dívidas

Nesse notebook pretendo programar um código que recebe uma tabela CSV e então realiza o calculo dos valores das parcelas por mês até finalizar a dívida.
Como forma de estudar a quantidade de meses que vai ser necessário para finaliza-la.

## Importações

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, date

from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.lib.units import cm
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import (
    SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle,
    Image, PageBreak
)
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

## Funções Auxiliares

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, date

from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.lib.units import cm
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import (
    SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle,
    Image, PageBreak
)
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont


# -----------------------------
# Helpers
# -----------------------------
def parse_brl_number(x):
    """Converte '1.287,36'/'1287,36'/'R$ 1.287,36' em float."""
    if pd.isna(x):
        return 0.0
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x).strip()
    s = s.replace("R$", "").replace(" ", "")
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return 0.0


def brl(v: float) -> str:
    """Formata float em BRL simples: 1234.5 -> 'R$ 1.234,50'."""
    v = float(v)
    s = f"{v:,.2f}"            # 1,234.50
    s = s.replace(",", "X").replace(".", ",").replace("X", ".")
    return f"R$ {s}"


def month_label(yyyy_mm: str) -> str:
    """'2026-02' -> 'fev/2026'"""
    try:
        y, m = yyyy_mm.split("-")
        m = int(m)
        meses = ["jan", "fev", "mar", "abr", "mai", "jun",
                 "jul", "ago", "set", "out", "nov", "dez"]
        return f"{meses[m-1]}/{y}"
    except Exception:
        return yyyy_mm


def safe_read_csv(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        raise FileNotFoundError(f"Arquivo não encontrado: {path}")
    return pd.read_csv(path, sep=";", dtype=str, keep_default_na=False)


def header_footer(canvas, doc, title: str):
    canvas.saveState()
    w, h = A4

    # Header
    canvas.setFillColor(colors.HexColor("#111827"))
    canvas.setFont("Helvetica-Bold", 10)
    canvas.drawString(2*cm, h - 1.2*cm, title)

    # Footer
    canvas.setFont("Helvetica", 9)
    canvas.setFillColor(colors.HexColor("#6B7280"))
    canvas.drawString(2*cm, 1.0*cm, f"Gerado em {datetime.now().strftime('%d/%m/%Y %H:%M')}")
    canvas.drawRightString(w - 2*cm, 1.0*cm, f"Página {doc.page}")

    canvas.restoreState()


def make_table(data, col_widths=None, header_bg="#111827"):
    tbl = Table(data, colWidths=col_widths, repeatRows=1)

    tbl_style = TableStyle([
        ("BACKGROUND", (0, 0), (-1, 0), colors.HexColor(header_bg)),
        ("TEXTCOLOR", (0, 0), (-1, 0), colors.white),
        ("FONTNAME", (0, 0), (-1, 0), "Helvetica-Bold"),
        ("FONTSIZE", (0, 0), (-1, 0), 10),
        ("ALIGN", (0, 0), (-1, 0), "CENTER"),
        ("VALIGN", (0, 0), (-1, -1), "MIDDLE"),

        ("FONTNAME", (0, 1), (-1, -1), "Helvetica"),
        ("FONTSIZE", (0, 1), (-1, -1), 9),

        ("GRID", (0, 0), (-1, -1), 0.25, colors.HexColor("#E5E7EB")),
        ("ROWBACKGROUNDS", (0, 1), (-1, -1), [colors.white, colors.HexColor("#F9FAFB")]),
        ("LEFTPADDING", (0, 0), (-1, -1), 6),
        ("RIGHTPADDING", (0, 0), (-1, -1), 6),
        ("TOPPADDING", (0, 0), (-1, -1), 5),
        ("BOTTOMPADDING", (0, 0), (-1, -1), 5),
    ])
    tbl.setStyle(tbl_style)
    return tbl


# -----------------------------
# Main PDF generator
# -----------------------------
def generate_finance_report_pdf(
    dividas_csv: str = "dividas.csv",
    mensal_csv: str = "pagamento_mensal.csv",
    output_pdf: str = "relatorio_financeiro_mensal.pdf",
    report_title: str = "Relatório Mensal — Dívidas Parceladas",
):
    # 1) Ler dados
    df_div = safe_read_csv(dividas_csv)
    df_m = safe_read_csv(mensal_csv)

    # 2) Limpar/normalizar dívidas
    # Ajuste de nomes conforme seu CSV
    col_name = "Nome da Dívida"
    col_total = "Valor Total"
    col_parc_total = "Nº de Parcelas"
    col_parc_value = "Valor da Parcela"
    col_paid = "Parcelas Pagas"
    col_rest = "Valor Restante"
    col_status = "Status da Dívida"
    col_start = "Data de Início"
    col_obs = "Observações"

    # Converter numéricos
    for c in [col_total, col_parc_value, col_rest]:
        if c in df_div.columns:
            df_div[c] = df_div[c].apply(parse_brl_number)

    for c in [col_parc_total, col_paid]:
        if c in df_div.columns:
            df_div[c] = pd.to_numeric(df_div[c], errors="coerce").fillna(0).astype(int)

    # Filtrar ativas (se existir)
    if col_status in df_div.columns:
        df_div["__status_norm"] = df_div[col_status].astype(str).str.strip().str.lower()
        df_div_ativas = df_div[df_div["__status_norm"] == "ativa"].copy()
    else:
        df_div_ativas = df_div.copy()

    df_div_ativas["parcelas_restantes"] = (df_div_ativas[col_parc_total] - df_div_ativas[col_paid]).clip(lower=0)

    # 3) Normalizar mensal
    if "mes" not in df_m.columns or "total_pago_no_mes" not in df_m.columns:
        raise ValueError("O CSV mensal precisa ter colunas: 'mes' e 'total_pago_no_mes'.")

    df_m["total_pago_no_mes"] = df_m["total_pago_no_mes"].apply(parse_brl_number)
    df_m = df_m.sort_values("mes")

    # 4) Métricas do relatório
    total_restante = df_div_ativas[col_rest].sum() if col_rest in df_div_ativas.columns else 0.0
    total_parcelas_restantes = int(df_div_ativas["parcelas_restantes"].sum())

    # mês atual (primeira linha do cronograma, se existir)
    if len(df_m) > 0:
        mes_atual = df_m.iloc[0]["mes"]
        pago_mes_atual = float(df_m.iloc[0]["total_pago_no_mes"])
        mes_final = df_m.iloc[-1]["mes"]
        meses_restantes = len(df_m)
        total_a_pagar_ate_fim = float(df_m["total_pago_no_mes"].sum())
    else:
        mes_atual = datetime.now().strftime("%Y-%m")
        pago_mes_atual = 0.0
        mes_final = mes_atual
        meses_restantes = 0
        total_a_pagar_ate_fim = 0.0

    # 5) Gráfico mensal (matplotlib -> PNG)
    chart_path = "grafico_pagamento_mensal.png"
    if len(df_m) > 0:
        plt.figure()
        x = [month_label(m) for m in df_m["mes"].tolist()]
        y = df_m["total_pago_no_mes"].tolist()
        plt.plot(x, y, marker="o")
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        plt.ylabel("Total pago no mês")
        plt.title("Evolução do pagamento mensal")
        plt.savefig(chart_path, dpi=160)
        plt.close()
    else:
        chart_path = None

    # 6) Montar PDF (ReportLab)
    doc = SimpleDocTemplate(
        output_pdf,
        pagesize=A4,
        leftMargin=2*cm,
        rightMargin=2*cm,
        topMargin=2*cm,
        bottomMargin=2*cm
    )

    styles = getSampleStyleSheet()
    styles.add(ParagraphStyle(
        name="TitleBig",
        parent=styles["Title"],
        fontName="Helvetica-Bold",
        fontSize=20,
        textColor=colors.HexColor("#111827"),
        spaceAfter=12
    ))
    styles.add(ParagraphStyle(
        name="Subtle",
        parent=styles["Normal"],
        fontSize=10,
        textColor=colors.HexColor("#6B7280"),
        leading=14
    ))
    styles.add(ParagraphStyle(
        name="H2",
        parent=styles["Heading2"],
        fontName="Helvetica-Bold",
        fontSize=13,
        textColor=colors.HexColor("#111827"),
        spaceBefore=12,
        spaceAfter=8
    ))
    styles.add(ParagraphStyle(
        name="CardLabel",
        parent=styles["Normal"],
        fontName="Helvetica-Bold",
        fontSize=9,
        textColor=colors.HexColor("#6B7280"),
        leading=12
    ))
    styles.add(ParagraphStyle(
        name="CardValue",
        parent=styles["Normal"],
        fontName="Helvetica-Bold",
        fontSize=14,
        textColor=colors.HexColor("#111827"),
        leading=16
    ))

    story = []

    # Capa / Cabeçalho
    story.append(Paragraph(report_title, styles["TitleBig"]))
    story.append(Paragraph(
        f"Mês de referência: <b>{month_label(mes_atual)}</b>  •  Projeção até: <b>{month_label(mes_final)}</b>",
        styles["Subtle"]
    ))
    story.append(Spacer(1, 12))

    # “Cards” de resumo (tabela estilizada)
    cards = [
        ["PAGO NO MÊS ATUAL", "MESES RESTANTES", "TOTAL RESTANTE (SOMA)", "TOTAL A PAGAR (CRONOGRAMA)"],
        [brl(pago_mes_atual), str(meses_restantes), brl(total_restante), brl(total_a_pagar_ate_fim)]
    ]
    card_tbl = Table(cards, colWidths=[(A4[0]-4*cm)/4]*4)
    card_tbl.setStyle(TableStyle([
        ("BACKGROUND", (0,0), (-1,0), colors.HexColor("#F3F4F6")),
        ("TEXTCOLOR", (0,0), (-1,0), colors.HexColor("#6B7280")),
        ("FONTNAME", (0,0), (-1,0), "Helvetica-Bold"),
        ("FONTSIZE", (0,0), (-1,0), 9),
        ("ALIGN", (0,0), (-1,0), "CENTER"),
        ("BOTTOMPADDING", (0,0), (-1,0), 8),
        ("TOPPADDING", (0,0), (-1,0), 8),

        ("BACKGROUND", (0,1), (-1,1), colors.white),
        ("FONTNAME", (0,1), (-1,1), "Helvetica-Bold"),
        ("FONTSIZE", (0,1), (-1,1), 14),
        ("TEXTCOLOR", (0,1), (-1,1), colors.HexColor("#111827")),
        ("ALIGN", (0,1), (-1,1), "CENTER"),
        ("BOTTOMPADDING", (0,1), (-1,1), 10),
        ("TOPPADDING", (0,1), (-1,1), 10),

        ("BOX", (0,0), (-1,-1), 0.8, colors.HexColor("#E5E7EB")),
        ("INNERGRID", (0,0), (-1,-1), 0.5, colors.HexColor("#E5E7EB")),
    ]))
    story.append(card_tbl)
    story.append(Spacer(1, 14))

    story.append(Paragraph("Resumo rápido", styles["H2"]))
    story.append(Paragraph(
        f"- Parcelas restantes (somadas): <b>{total_parcelas_restantes}</b><br/>"
        f"- Se você mantiver esse cronograma, a quitação estimada é em: <b>{month_label(mes_final)}</b><br/>"
        f"- Observação: o cronograma mensal reflete sua regra de geração (ex.: começar no mês atual).",
        styles["Normal"]
    ))
    story.append(Spacer(1, 10))

    # Gráfico
    if chart_path and os.path.exists(chart_path):
        story.append(Paragraph("Pagamento mensal (gráfico)", styles["H2"]))
        story.append(Image(chart_path, width=16.5*cm, height=8.5*cm))
        story.append(Spacer(1, 8))

    # Tabela mensal
    story.append(Paragraph("Cronograma mensal de pagamento", styles["H2"]))
    mensal_table_data = [["Mês", "Total a pagar no mês"]]
    for _, r in df_m.iterrows():
        mensal_table_data.append([month_label(r["mes"]), brl(r["total_pago_no_mes"])])
    story.append(make_table(mensal_table_data, col_widths=[6*cm, 10.5*cm]))
    story.append(Spacer(1, 10))

    # Tabela das dívidas
    story.append(PageBreak())
    story.append(Paragraph("Dívidas ativas (detalhamento)", styles["H2"]))

    # Selecionar colunas úteis, mantendo robusto se faltar alguma
    cols = [
        col_name, col_total, col_parc_total, col_parc_value, col_paid,
        "parcelas_restantes", col_rest, col_start, col_obs
    ]
    cols = [c for c in cols if c in df_div_ativas.columns]

    # Monta linhas
    div_table_data = [[
        "Dívida", "Total", "Parc.", "R$/Parc.", "Pagas", "Rest.", "Saldo", "Início", "Obs."
    ]]

    for _, r in df_div_ativas.iterrows():
        div_table_data.append([
            str(r.get(col_name, ""))[:28],
            brl(r.get(col_total, 0)),
            str(r.get(col_parc_total, "")),
            brl(r.get(col_parc_value, 0)),
            str(r.get(col_paid, "")),
            str(r.get("parcelas_restantes", "")),
            brl(r.get(col_rest, 0)),
            str(r.get(col_start, "")),
            str(r.get(col_obs, ""))[:30],
        ])

    # Larguras equilibradas
    col_widths = [4.2*cm, 2.2*cm, 1.2*cm, 2.2*cm, 1.2*cm, 1.2*cm, 2.2*cm, 2.0*cm, 3.2*cm]
    story.append(make_table(div_table_data, col_widths=col_widths))
    story.append(Spacer(1, 10))

    story.append(Paragraph(
        "Dica: se alguma dívida estiver com Data de Início indefinida, o cronograma pode não refletir o mês real; "
        "neste caso, preencha a data e gere novamente o CSV mensal.",
        styles["Subtle"]
    ))

    # Build
    doc.build(
        story,
        onFirstPage=lambda c, d: header_footer(c, d, report_title),
        onLaterPages=lambda c, d: header_footer(c, d, report_title),
    )

    # Limpa imagem temporária
    if chart_path and os.path.exists(chart_path):
        try:
            os.remove(chart_path)
        except OSError:
            pass

    print(f"✅ PDF gerado: {output_pdf}")

## Área de Execução

In [3]:
%%time
generate_finance_report_pdf(
    dividas_csv="dividas.csv",
    mensal_csv="pagamento_mensal.csv",
    output_pdf="relatorio_financeiro_mensal.pdf",
    report_title="Relatório Mensal — Dívidas Parceladas"
)

✅ PDF gerado: relatorio_financeiro_mensal.pdf
CPU times: total: 203 ms
Wall time: 295 ms
